In [ ]:
!pip install -Uqq gradio
!pip install -Uqq nbdev

In this notebook we will use nbdev library to export notebook to python script. So we can first check results interactively and then swiftly move code to the regular python script.

`#|default_exp app` - tells the nbdev export functionality to put output script to the app directory.
`#|export` - marks cells taht should be exported

In [ ]:
#|default_exp app

In [ ]:
#|export
import gradio as gr
from fastai.vision.all import *

## Run the model predict on saved checkpoint
Let's get some beer images from the duck duck go and run predictions with saved model

In [ ]:
from IPython import display

from duckduckgo_search import ddg_images
from fastcore.all import L
from fastdownload import download_url

In [ ]:


def search_images(term, max_images=200): 
    return L(ddg_images(term, max_results=max_images)).itemgot('image')

for id_ in ["harnas", "perla", "warka"]:
    urls = search_images(f'piwo {id_}', max_images=1)
    dest = f'{id_}.jpg'
    download_url(urls[0], dest, show_progress=False)
    im = Image.open(dest)
    im.to_thumb(192, 192)
    display.display(im)

In [ ]:
#|export
model = load_learner("./export.pkl")

In [ ]:
model.predict(im)

## Use GradIO to create interactive app
First we will create a function that is an entrypoint to run predictions with the model
Then we will create GradIO interface that runs this entrypoint on provided Image inputs and examples
We will visualize locally if everything works as expected

In [ ]:
#|export

categories = ['harnas', 'perla', 'warka']
def classify_beer(img):
    pred, idx, probs = model.predict(img)
    return dict(zip(categories, map(float, probs)))

In [ ]:
classify_beer(im)

In [ ]:
#|export

in_img = gr.Image(shape=(192, 192))
label = gr.Label()
examples = ["harnas.jpg", "perla.jpg", "warka.jpg"]

iface = gr.Interface(fn=classify_beer, inputs=in_img, outputs=label, examples=examples)
iface.launch(inline=False, debug=True, server_port=6060)

## Close all locally running gradio services

In [ ]:
gr.close_all()

## Create an app.py for HuggingFace space.

In [ ]:
import nbdev
nbdev.export.nb_export('lecture_2_deployment.ipynb', './app')

# Use GradIO client Python API with your model

In [ ]:
from gradio_client import Client
from pathlib import Path

client = Client("https://adam1105-ug-practical-deeplearning-fastai-2023.hf.space/")

def beer_predict_api(im_path):
    result_path = client.predict(im_path, # str (filepath or URL to image) in 'img' Image component
                        api_name="/predict")
    return json.loads(Path(result_path).read_text())

In [ ]:
beer_predict_api("perla.jpg")